<a href="https://colab.research.google.com/github/annu-creator24t/Model_training-_EEG/blob/main/LightGBM_Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas


In [2]:
pip install numpy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load the dataset
csv_path = "/content/drive/My Drive/Annu tiwari/Testing_Export_2_0.csv"
df = pd.read_csv(csv_path)

# Drop unnecessary columns
df = df.drop(columns=["File", "Channel", "DFA"], errors="ignore")

# Shuffle the dataset
df = df.sample(frac=1, random_state=82).reset_index(drop=True)

# Define features (X) and target (y)
X = df.drop(columns=["Patient_Type"])  # Features
y = df["Patient_Type"]  # Target (0 = Healthy, 1 = Depressed)

# Apply feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train LightGBM Model
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)

# Set optimized parameters for LightGBM
params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.03,
    'num_leaves': 50,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Train the model
lgb_model = lgb.train(params, lgb_train, num_boost_round=200, valid_sets=[lgb_test])

# Make predictions
y_pred_lgb = (lgb_model.predict(X_test) > 0.5).astype(int)

# Function to print evaluation metrics
def evaluate_model(name, y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy
    print(f"==== {name} ====")
    print(f"Model Accuracy: {accuracy * 100:.2f}%")  # Print accuracy in percentage
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, digits=4))

# Evaluate LightGBM Model
evaluate_model("LightGBM", y_test, y_pred_lgb)



==== LightGBM ====
Model Accuracy: 94.29%

Confusion Matrix:
[[161  14]
 [  6 169]]

Classification Report:
              precision    recall  f1-score   support

           0     0.9641    0.9200    0.9415       175
           1     0.9235    0.9657    0.9441       175

    accuracy                         0.9429       350
   macro avg     0.9438    0.9429    0.9428       350
weighted avg     0.9438    0.9429    0.9428       350

